# Process GWPT dataset from raw to clean.

In [11]:
import pandas as pd
import plotly.express as px

## Read in the data into a single dataframe
Join the small and large dataset (> 10MW and <10MW) together.

In [12]:
# set up parameters
INPUT_FILE = r"..\data\raw\Global-Wind-Power-Tracker-December-2023.xlsx"
WORKSHEET_BIG_DATASET = "Data"
WORKSHEET_SMALL_DATASET = "Below Threshold"
OUTPUT_FILE_FULL_DATASET = r"..\data\clean\gwpt.parquet"
OUTPUT_FILE_GEO = r"..\data\clean\geo.parquet"
OUTPUT_FILE_AGG = r"..\data\clean\gwpt_agg.parquet"

In [13]:
# read in the big & small dataset
df_big = pd.read_excel(INPUT_FILE, sheet_name=WORKSHEET_BIG_DATASET)
df_small = pd.read_excel(INPUT_FILE, sheet_name=WORKSHEET_SMALL_DATASET)
print(f"The dimensions of the big table are {df_big.shape}")
print(f"The dimensions of the small table are {df_small.shape}")

The dimensions of the big table are (26523, 29)
The dimensions of the small table are (899, 29)


In [14]:
#combining them 
df = pd.concat([df_big, df_small])
df.head(5)

,Date Last Researched,Country,Project Name,Phase Name,Project Name in Local Language / Script,Other Name(s),Capacity (MW),Installation Type,Status,Start year,...,"Local area (taluk, county)","Major area (prefecture, district)",State/Province,Subregion,Region,GEM location ID,GEM phase ID,Other IDs (location),Other IDs (unit/phase),Wiki URL
0,2023/07/03,Algeria,Kabertene wind farm,NaN,"مزرعة رياح كبيرت,, مدينة أدرار",Kabartene wind farm,10.0,onshore,operating,2014.0,...,Tsabit District,NaN,Adrar,Northern Africa,Africa,L900124,G900162,NaN,NaN,https://gem.wiki/Kabertene_wind_farm
1,2023/07/03,Algeria,Khenchela wind farm,NaN,مزرعة رياح خنشلة,NaN,20.0,onshore,cancelled,NaN,...,NaN,NaN,NaN,Northern Africa,Africa,L900137,G900178,NaN,NaN,https://gem.wiki/Khenchela_wind_farm
2,2023/07/03,Algeria,Timimoun wind farm,NaN,NaN,Timimoun Sktm,50.0,unknown,cancelled,NaN,...,NaN,NaN,NaN,Northern Africa,Africa,L916594,G920620,NaN,NaN,https://gem.wiki/Timimoun_wind_farm
3,2023/07/06,Angola,Benjamin wind farm,NaN,NaN,NaN,52.0,onshore,pre-construction,2028.0,...,NaN,NaN,Benguela Province,Sub-Saharan Africa,Africa,L916595,G920621,NaN,NaN,https://gem.wiki/Benjamin_wind_farm
4,2023/07/06,Angola,Cacula wind farm,NaN,NaN,NaN,88.0,onshore,pre-construction,2029.0,...,NaN,NaN,Huíla Province,Sub-Saharan Africa,Africa,L916596,G920622,NaN,NaN,https://gem.wiki/Cacula_wind_farm


## Explore the data
### Fix datatypes

In [5]:
# check and correct the types of the imported data
df.dtypes

Date Last Researched                         object
Country                                      object
Project Name                                 object
Phase Name                                   object
Project Name in Local Language / Script      object
Other Name(s)                                object
Capacity (MW)                               float64
Installation Type                            object
Status                                       object
Start year                                  float64
Retired year                                float64
Operator                                     object
Operator Name in Local Language / Script     object
Owner                                        object
Owner Name in Local Language / Script        object
Latitude                                    float64
Longitude                                   float64
Location accuracy                            object
City                                         object
Local area (

In [6]:
# convert the date columns which are in object (string) to date types
df["Date Last Researched"] = pd.to_datetime(df["Date Last Researched"], format='ISO8601') 

In [7]:
# convert the float columns which are whole numbers to ints
df['Start year'] = df['Start year'].astype('Int64')
df['Retired year'] = df['Retired year'].astype('Int64')

### Look for missing values


In [32]:
missing_data = df.isna().sum()
missing_data

Date Last Researched                            0
Country                                         0
Project Name                                    0
Phase Name                                  16831
Project Name in Local Language / Script     15849
Other Name(s)                               21906
Capacity (MW)                                   0
Installation Type                               0
Status                                          0
Start year                                   9919
Retired year                                27156
Operator                                    10210
Operator Name in Local Language / Script    20932
Owner                                        7345
Owner Name in Local Language / Script       21411
Latitude                                        0
Longitude                                       0
Location accuracy                               0
City                                        15172
Local area (taluk, county)                   9059


In [28]:
# missing capacity:
capacity_na_count = df['Capacity (MW)'].isna().sum()
print(f"Number of rows with NaN in 'Capacity (MW)': {capacity_na_count}")

Number of rows with NaN in 'Capacity (MW)': 0


In [27]:
# missing start_year
start_na_count = df['Start year'].isna().sum()
print(f"Number of rows with NaN in 'Start year': {start_na_count}")
print(f"Percentage of rows with 'Start year' info: {(df.shape[0]-start_na_count)/df.shape[0]*100} %")

Number of rows with NaN in 'Start year': 9919
Percentage of rows with 'Start year' info: 63.8283130333309 %


In [67]:
# missing region:
region_na_count = df['Region'].isna().sum()
print(f"Number of rows with NaN in 'Region': {region_na_count}")

Number of rows with NaN in 'Region': 0


In [68]:
# missing country:
country_na_count = df['Country'].isna().sum()
print(f"Number of rows with NaN in 'Country': {country_na_count}")

Number of rows with NaN in 'Country': 0


In [69]:
# missing coordinates:
coord_na_count = df[df['Latitude'].isna() | df['Longitude'].isna()].shape[0]
print(f"Number of rows with NaN in 'Lat or Lon': {coord_na_count}")

Number of rows with NaN in 'Lat or Lon': 0


### Check categorical columns

In [70]:
# check country
unique_countries = df["Country"].unique()
print("The distinct values for 'Country' are:")
print(unique_countries)
print(f"There are {len(unique_countries)} unique countries in the dataset.")

The distinct values for 'Country' are:
['Algeria' 'Angola' 'Cabo Verde' 'Cameroon' 'Chad' 'Djibouti' 'Egypt'
 'Ethiopia' 'Ghana' 'Kenya' 'Libya' 'Madagascar' 'Malawi' 'Mali'
 'Mauritania' 'Mauritius' 'Morocco' 'Mozambique' 'Namibia' 'Niger'
 'Nigeria' 'Réunion' 'Senegal' 'South Africa' 'Sudan' 'Tanzania' 'Togo'
 'Tunisia' 'Uganda' 'Western Sahara' 'Zambia' 'Zimbabwe' 'Argentina'
 'Aruba' 'Barbados' 'Bermuda' 'Bolivia' 'Bonaire, Sint Eustatius and Saba'
 'Brazil' 'Canada' 'Chile' 'Colombia' 'Costa Rica' 'Cuba' 'Curaçao'
 'Dominican Republic' 'Ecuador' 'El Salvador' 'Greenland' 'Grenada'
 'Guadeloupe' 'Guatemala' 'Guyana' 'Honduras' 'Jamaica' 'Martinique'
 'Mexico' 'Nicaragua' 'Panama' 'Peru' 'Puerto Rico' 'Saint Lucia'
 'United States' 'Uruguay' 'Venezuela' 'Vietnam' 'South Korea' 'Jordan'
 'Philippines' 'Japan' 'Armenia' 'Pakistan' 'Türkiye' 'Cyprus' 'Israel'
 'Iran' 'Kazakhstan' 'Saudi Arabia' 'Syria' 'Yemen' 'Indonesia' 'Oman'
 'Thailand' 'Myanmar' 'Sri Lanka' 'Azerbaijan' 'Uzbekista

In [71]:
# check Installation Type
unique_installation_types = df["Installation Type"].unique()
print("The distinct values for 'Installation Type' are:")
print(unique_installation_types)

The distinct values for 'Installation Type' are:
['onshore' 'unknown' 'offshore mount unknown' 'offshore hard mount'
 'offshore floating']


In [72]:
# check status Type
unique_status = df["Status"].unique()
print("The distinct values for 'Status' are:")
print(unique_status)

The distinct values for 'Status' are:
['operating' 'cancelled' 'pre-construction' 'announced' 'construction'
 'shelved' 'retired' 'mothballed']


In [73]:
# check Installation Type
unique_region = df["Region"].unique()
print("The distinct values for 'Region' are:")
print(unique_region)

The distinct values for 'Region' are:
['Africa' 'Americas' 'Asia' 'Europe' 'Oceania']


## Clean the data
### Drop columns you will never use

In [74]:
columns_to_drop = ["Project Name in Local Language / Script", 
                   "Owner Name in Local Language / Script",
                   "GEM location ID",
                   "GEM phase ID",
                   "Other IDs (location)",
                   "Other IDs (unit/phase)",
                   "Other Name(s)",
                   "Date Last Researched"
                   ]
df = df.drop(columns=columns_to_drop)

### Drop rows you will never use (outliers that are proven wrong and will mess up the viz)

In [75]:
df = df[df["Capacity (MW)"] < 10000] 

## Make first visuals

In [76]:
# stacked bar chart of installed capacity per region, colored by type

# Create a stacked bar chart
fig = px.bar(df, x='Region', y='Capacity (MW)', color='Status', title='Installed Capacity per Region')
fig.update_traces(marker_line_width=0)
fig.update_layout(barmode='stack')  # Set the barmode to 'stack' for stacking

# Show the plot
fig.show()

C:\dev\python\infovis\venv\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



## Create some aggregations (pre-processing) to speed up the later dash app

In [77]:
geo = df[["Region","Subregion","Country"]].drop_duplicates().reset_index()
geo = geo.reindex(columns=["Region", "Subregion", "Country"])
geo.shape

(155, 3)

In [78]:
agg = df.groupby(["Region", "Subregion", "Country", "Status", "Installation Type"]).sum("Capacity (MW)").reset_index()
agg.shape

(797, 10)

In [79]:
df["Capacity (MW)"].describe()
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Box(y=df["Capacity (MW)"], boxpoints='outliers', marker_color='skyblue', name="Capacity (MW)"))
fig.update_layout(title="Boxplot of Capacity (MW)",
                  yaxis_title="Capacity (MW)")
fig.show()

## Export to clean

In [80]:
df.to_parquet(OUTPUT_FILE_FULL_DATASET)
geo.to_parquet(OUTPUT_FILE_GEO)
agg.to_parquet(OUTPUT_FILE_AGG)

## Conclusion
### Closed issues
* Perform an initial assessment of the raw data to understand its structure, quality, and any potential issues.
* Identify missing values, outliers, duplicates, and inconsistencies within the data.
* Handle missing values through imputation or deletion based on the nature of the data and the analysis requirements.
* Standardize data formats, such as converting date/time formats, ensuring consistent units, and addressing encoding issues.
* Perform basic data transformations, such as normalization or scaling, if necessary.
* Document all data cleaning operations performed in this stage for transparency and reproducibility.
* Address outliers and anomalies by applying statistical methods or domain knowledge.

